In [ ]:
import setup
setup.init_django(project_name='config')

In [ ]:
import helpers

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
url = "https://www.amazon.com.br/Barra-Prote%C3%ADna-BOLD-Crunch-Cookies/dp/B0D5JJ2KDP/ref=pd_rhf_dp_s_pd_crcd_d_sccl_2_6/145-2538442-3375966?pd_rd_w=emFuO&content-id=amzn1.sym.1b2190ca-db28-46be-85f8-55611bc8c041&pf_rd_p=1b2190ca-db28-46be-85f8-55611bc8c041&pf_rd_r=ME15A3221VFE4TDDADFT&pd_rd_wg=JJ6ft&pd_rd_r=c9fd334d-45f2-4371-8198-0f2e370dd11b&pd_rd_i=B0D5JJ2KDP&psc=1"

In [ ]:
url

In [ ]:
html = helpers.scrape(url=url, solve_captcha=False)
data = helpers.extract_amazon_product_data(html)

In [ ]:
# data

In [ ]:
def find_product_table_data(html):
    soup = BeautifulSoup(html)
    product_data = soup.find('div', id='prodDetails')
    if product_data is None:
        return []
    table = product_data.find('table')
    columns = [f"{x.text}".strip() for x in table.find_all('th')]
    table_data=[]
    for i, row in enumerate(table.find_all('tr')):  # [1:] to skip the header row
        # Get all cells in the row
        cells = row.find_all('td')
        # Create a dictionary for the current row, mapping header to cell data
        row_data = {columns[i]: f'{cell.text}'.strip() for cell in cells}
        # Add the dictionary to your list
        table_data.append(row_data)
    return table_data

def find_product_rating(html):
    soup = BeautifulSoup(html)
    average_rating = soup.find(id='averageCustomerReviews').find_all("span", class_='a-size-base')[0].text.strip()
    average_rating = "".join([x for x in f"{average_rating}".strip() if x.isdigit() or x == ','])
    average_rating = float(average_rating.replace(',', '.'))
    rating_data = soup.find(id='acrCustomerReviewText').text
    rating_count = int(''.join([x for x in rating_data if x.isdigit()]))
    rating_count
    return {
        'average': average_rating,
        'count': rating_count,
    }

def extract_amazon_product_data(html):
    soup = BeautifulSoup(html)
    productTitle = soup.find('span', id='productTitle')
    productTitleText = f"{productTitle.text}".strip()
    productPrice = soup.find_all('span', class_='a-price-whole')[0]
    productPrice = f"{productPrice.text}".strip()
    productPriceText = "".join([x for x in productPrice if x.isdigit() or x == ','])
    productPriceNum = float(productPriceText)
    try:
        productDescription = soup.find('div', id='productDescription').text
    except:
        productDescription = ''
    featureBullets = soup.find('div', id='feature-bullets').text
    return {
        'title': productTitleText,
        'price_raw': productPrice,
        'price_text': productPriceText,
        'price': productPriceNum,
        'metadata': find_product_table_data(html),
        'description': productDescription,
        'feature_bullets': featureBullets,
        'rating': find_product_rating(html)
    }

In [ ]:
html = helpers.scrape(url=url, solve_captcha=False)

In [ ]:
data = extract_amazon_product_data(html)
data